In [145]:
import torch
import numpy as np
import pandas as pd

train_X = pd.read_csv('X_train.csv')
train_X_tensor = torch.tensor(train_X.values)[:,1:]
train_y = pd.read_csv("y_train.csv")
train_y_tensor = torch.tensor(train_y.values)[:,1:]

test_X = pd.read_csv('X_test.csv')
test_X_tensor = torch.tensor(test_X.values)[:,1:]

In [129]:
train_X.shape

(1212, 833)

---------------------------------------------FIRST IMPUTATION-------------------------------------------

In [146]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10, weights="uniform")
imputer.fit(train_X_tensor.numpy())

train_X_knn_first = imputer.transform(train_X_tensor.numpy())

In [147]:
train_y_knn_first = train_y_tensor.numpy()
print(train_X_knn_first.shape)
print(train_y_knn_first.shape)

(1212, 832)
(1212, 1)


------------------------------------------------FIRST OUTLIER DETECTION------------------------------------------------

In [148]:
from sklearn.ensemble import IsolationForest
clf_isoforest = IsolationForest(random_state=0, bootstrap = True)
first_iso_outliers = clf_isoforest.fit_predict(train_X_knn_first)

In [149]:
count = 0
for x in range(len(first_iso_outliers)):
  if first_iso_outliers[x]!=1:
    print(x)
    count +=1
print(f"count is {count}")
first_iso_outliers+=1
first_iso_outliers = first_iso_outliers/2
train_X_iso_first = train_X_knn_first[first_iso_outliers==1.0]
train_y_iso_first = train_y_knn_first[first_iso_outliers==1.0]

40
118
239
426
427
565
652
670
732
997
count is 10


In [152]:
train_X_iso_first.shape

(1202, 832)

In [153]:
train_y_iso_first.shape

(1202, 1)

------------------------------------------------FIRST FEATURE SELECTION (RF)----------------------------------------------

In [154]:
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor(n_estimators=100, random_state=0,oob_score=True, bootstrap=True)
rf1.fit(train_X_iso_first, train_y_iso_first)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=0, verbose=0, warm_start=False)

In [156]:
train_X_features_first = train_X_iso_first[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]]
train_y_features_first = train_y_iso_first
print(train_X_features_first.shape)
print(train_y_features_first.shape)

(1202, 256)
(1202, 1)


-------------------SHOULD I INCLUDE SECOND IMPUTATION?-------------------------


In [65]:
# imputer_second = KNNImputer(n_neighbors=10, weights="uniform")
# imputer_second.fit((train_X_tensor.numpy()[first_iso_outliers==1.0])[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]])

# train_X_knn_second = imputer_second.transform((train_X_tensor.numpy()[first_iso_outliers==1.0])[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]])
# train_y_knn_second = train_y_features_first

------------------------------------------------------SECOND OUTLIER DETECTION (isoforest)-----------------------------------------------------

In [157]:
clf_isoforest2 = IsolationForest(random_state=0, bootstrap = True)
second_iso_outliers = clf_isoforest2.fit_predict(train_X_features_first)

In [158]:
count = 0
for x in range(len(second_iso_outliers)):
  if second_iso_outliers[x]!=1:
    print(x)
    count +=1
print(f"count is {count}")
second_iso_outliers+=1
second_iso_outliers = second_iso_outliers/2
train_X_iso_second = train_X_features_first[second_iso_outliers==1.0]
train_y_iso_second = train_y_features_first[second_iso_outliers==1.0]

1
3
14
16
34
39
40
41
93
109
132
140
141
144
166
203
213
237
276
310
320
328
330
345
390
426
433
482
516
554
565
567
579
590
591
603
608
621
650
652
665
670
694
738
756
765
780
783
817
819
826
837
839
862
872
875
909
924
939
972
984
998
1084
1091
1094
1097
1100
1133
1155
1169
1192
count is 71


In [159]:
print(train_X_iso_second.shape)
print(train_y_iso_second.shape)

(1131, 256)
(1131, 1)


HERE, ADD AN EXTRA STEP OF THE ENTIRE PROCESS ABOVE

-------------------------------------------------------REGRESSION---------------------------------------------------

In [160]:
from sklearn.metrics import r2_score
from sklearn.ensemble import BaggingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

In [161]:
#Split to validation and training set
train_X_iso_second_a = train_X_iso_second[:1040]
train_X_iso_second_b = train_X_iso_second[1040:]
train_y_iso_second_a = train_y_iso_second[:1040]
train_y_iso_second_b = train_y_iso_second[1040:]

In [163]:
gbt = make_pipeline(StandardScaler(),GradientBoostingRegressor(n_estimators=345, learning_rate=0.14, random_state=0))
gbt.fit(train_X_iso_second_a, train_y_iso_second_a)
r2_score(train_y_iso_second_b, gbt.predict(train_X_iso_second_b))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6140776163627466

In [164]:
gbt2 = make_pipeline(StandardScaler(),GradientBoostingRegressor(n_estimators=345, learning_rate=0.14, random_state=0))
gbt2.fit(train_X_iso_second, train_y_iso_second)
r2_score(train_y_iso_second, gbt2.predict(train_X_iso_second))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9937361767779619

-------------------------------------------------SECOND FEATURE SELECTION-----------------------------------------------

In [49]:
rf2 = RandomForestRegressor(n_estimators=100, random_state=0,oob_score=True, bootstrap=True)
rf2.fit(train_X_iso_second, train_y_iso_second)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=0, verbose=0, warm_start=False)

In [50]:
train_X_features_second = train_X_iso_second[:,rf2.feature_importances_>=sorted(rf2.feature_importances_)[-256]]
train_y_features_second = train_y_iso_second
print(train_X_features_second.shape)
print(train_y_features_second.shape)

(1131, 256)
(1131, 1)


----------------------------------------------THIRD OUTLIER DETECTION-------------------------------------------

In [51]:
clf_isoforest3 = IsolationForest(random_state=0, bootstrap = True)
third_iso_outliers = clf_isoforest3.fit_predict(train_X_features_second)

In [ ]:
count = 0
for x in range(len(third_iso_outliers)):
  if third_iso_outliers[x]!=1:
    print(x)
    count +=1
print(f"count is {count}")
third_iso_outliers+=1
third_iso_outliers = third_iso_outliers/2
train_X_iso_third = train_X_features_second[third_iso_outliers==1.0]
train_y_iso_third = train_y_features_second[third_iso_outliers==1.0]

In [53]:
print(train_X_iso_third.shape)
print(train_y_iso_third.shape)

(1084, 256)
(1084, 1)


--------------------------------------------REGRESSION (after 3 outlier steps)---------------------------------------------

In [54]:
train_X_iso_third_a = train_X_iso_third[:1010]
train_X_iso_third_b = train_X_iso_third[1010:]
train_y_iso_third_a = train_y_iso_third[:1010]
train_y_iso_third_b = train_y_iso_third[1010:]

In [55]:
gbt_third = make_pipeline(StandardScaler(),GradientBoostingRegressor(n_estimators=350, learning_rate=0.14, random_state=0))
gbt_third.fit(train_X_iso_third_a, train_y_iso_third_a)
r2_score(train_y_iso_third_b, gbt_third.predict(train_X_iso_third_b))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5738980398414222

In [43]:
gbt_third2 = make_pipeline(StandardScaler(),GradientBoostingRegressor(n_estimators=350, learning_rate=0.14, random_state=0))
gbt_third2.fit(train_X_iso_third, train_y_iso_third)
r2_score(train_y_iso_third, gbt_third2.predict(train_X_iso_third))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9947555415969989

--------------------------------------------------MAKING PREDICTIONS------------------------------------------------

In [44]:
imputertest = KNNImputer(n_neighbors=10, weights="uniform") #Should I use the already trained imputer?

imputertest.fit(test_X_tensor.numpy())

test_X_knn = imputertest.transform(test_X_tensor.numpy())

In [99]:
test_GAIN = test_X_tensor.numpy()

In [47]:
temp = test_X_knn[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]]
temp2 = temp[:,rf2.feature_importances_>=sorted(rf2.feature_importances_)[-250]]
predictiontest = gbt_third2.predict(temp2)

In [100]:
predictiontest = gbt2.predict(test_GAIN[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]])

In [ ]:
predictiontest = gbt2.predict(test_X_knn[:,rf1.feature_importances_>=sorted(rf1.feature_importances_)[-256]])

In [101]:
test_y = pd.read_csv('sample.csv')
test_y.loc[:,"y"] = predictiontest
test_y.to_csv("predictions4.csv", index=False)